<a href="https://colab.research.google.com/github/jansoe/KISchule/blob/main/A7_2_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7. Transformer

Innerhalb von A7_0 und A7_1 haben Sie mit unterschiedlichen Varianten von BERT-Netzwerken gearbeitet, die für NLP-Problemstellungen trainiert worden sind. Jetzt widmen wir uns in ähnlicher Weise den Text generierenden Modellen der GPT-2 Familie: Zunächst betrachten wir Beispiele von trainierten GPT-2-Netzen. Im anschließenden Notebook A7_3 können Sie dann versuchen, die Fähigkeiten eines GPT-2-Modells über geziehltes Nachtrainieren an speziellere Anforderungen anzupassen.

## 7.2 Textgenerierung mit GPT-2

### 7.2.0 Vorbereitungen

In [ ]:
%tensorflow_version 2
!pip install transformers==2.4
import transformers
from pprint import pprint

### 7.2.1 Das Modell

Hier können Sie das Modell auswählen, mit dem Text generiert werden soll. Am besten geeignet ist zur Zeit die GPT-2 Familie, von der es unterschiedlich komplexe Modelle gibt: `gpt2`, `gpt2-medium`, `gpt2-large`, `gpt2-xl`.

In [ ]:
model_name_or_path = 'gpt2'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)
model = transformers.AutoModelWithLMHead.from_pretrained(model_name_or_path)
_ = model.to('cuda') # Modell auf GPU schieben

Als nächsten wählen Sie einen Text, mit dem das Modell "angefüttert" wird ...

In [ ]:
prompt_text = 'What is artificial Intelligence? It '

... und tokenisieren diesen Text.

In [ ]:
encoded_prompt = tokenizer.encode(prompt_text, return_tensors="pt")
encoded_prompt = encoded_prompt.to('cuda')

Der Textgenerierung liegt zugrunde, dass das Modell in jedem Schritt für jedes Token im Vokabular eine Wahrscheinlichkeit angibt, mit der das Modell das jeweilige Token für die nächste Position vorhersagt. Nun können wir 'greedy' einfach immer das wahrscheinlichste Token auswählen und zur nächsten Position schreiten. Aber dies führt oft zu durchwachsenen Resultaten (siehe https://arxiv.org/abs/1904.09751).

Für eine verbesserte Qualität der erzeugten Texte bieten sich verschiedene Einstellungsmöglichkeiten an:
- `repetition_penalty`: Tokens, die bereits generiert wurden, werden um diesen Faktor unwahrscheinlicher
- `do_sample`: Es wird nicht das wahrscheinlichste Token genommen sondern anhand der vorhergesagten Wahrscheinlichkeitsverteilung ein Token zufällig ausgewählt
  - `temperature`: Je höher die Temperatur, desto höher die Chance, dass auch unwahrscheinliche Tokens ausgewählt werden
  - `top_`: Einschränkungen für die zur Verfügung stehenden Tokens:
    - `top_k`: es werden nur die wahrscheinlichsten k Tokens zugelassen
    - `top_p`: von den wahrscheinlichsten Tokens werden so viele zugelassen, dass deren Gesamtwahrscheinlichkeit `p` nicht überschreitet.

Ausführlichere Informationen finden Sie auch [hier](https://towardsdatascience.com/how-to-sample-from-language-models-682bceb97277).

In [ ]:
output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_length=128, # Wieviele Token sollen maximal generiert werden
    repetition_penalty=3,
    do_sample=True,
    temperature=0.7,
    top_k= None, # set to None to ignore
    top_p=0.5, # set to None to ignore or 0 < p <= 1
)

In [ ]:
generated_sequence = output_sequences[0].tolist()
text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
pprint(text)

### 7.2.2 Aufgabe: Exploration der Parameter

Experimentieren Sie mit GPT-2 und den Modellfähigkeiten zur Textgenerierung. Wie können Sie die überzeugensten Texte generieren? Halten Sie Ihre Beobachtungen in geeigneter Weise hier im Notebook fest.

- Was beobachten Sie, wenn Sie `repetition_penalty = None` und `do_sample = False` setzten?
- Was passiert mit `do_sample = True` und verschiedenen Werten von `temperature` and `top_p`?
- Welchen Einfluss hat die Länge des Eingegebenen `prompt_text`?
- Was passiert, wenn Sie mit `prompt_text = '<|endoftext|>'` startet?

---
**Hinweis**

Bei `<|endoftext|>` handelt es sich um ein Spezialtoken, das GPT2 den Start eines neuen Dokuments signalisiert. Als prompt_text verwendet muss GPT2 somit ohne Kontext - also frei - Text generieren.

![insitubytes](https://drive.google.com/uc?id=1EAJK7AI9tcZRo3VvYq7vEKGxk7vmK2Ff)